In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [11]:
df_recipes = pd.read_csv("RAW_recipes.csv", converters={'tags': lambda x: x[1:-1].split(',')})  #https://stackoverflow.com/questions/45758646/pandas-convert-string-into-list-of-strings

In [12]:
#tags = df_recipes["tags"]#.iloc[0:1000]
df_recipes["tags"] = df_recipes["tags"].apply(lambda lst: [re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', s).lower() for s in lst])   #chatgpt for regex...

Counter(np.concatenate(df_recipes["tags"]))

Counter({'preparation': 230546,
         'time-to-make': 225326,
         'course': 218148,
         'main-ingredient': 170446,
         'dietary': 165091,
         'easy': 126062,
         'occasion': 114145,
         'cuisine': 91165,
         'low-in-something': 85776,
         'main-dish': 71786,
         'equipment': 70436,
         '60-minutes-or-less': 69990,
         'number-of-servings': 58949,
         'meat': 56042,
         '30-minutes-or-less': 55077,
         'vegetables': 53814,
         'taste-mood': 52143,
         '4-hours-or-less': 49497,
         'north-american': 48479,
         '3-steps-or-less': 44933,
         '15-minutes-or-less': 43934,
         'low-sodium': 43349,
         'desserts': 43203,
         'low-carb': 42189,
         'healthy': 40340,
         'dinner-party': 37561,
         'low-cholesterol': 36743,
         'low-calorie': 36429,
         'vegetarian': 35651,
         'beginner-cook': 35561,
         '5-ingredients-or-less': 35466,
         'holi

In [13]:
def filter_tags(df, tags):
    df_filtered = df.copy()
    tags_set = set(tags)
    df_filtered = df_filtered[~df_filtered['tags'].apply(lambda x: bool(tags_set.intersection(set(x))))]
    
    return df_filtered

df_temp = filter_tags(df_recipes,["beverages"])

## Ingredients extraction
#### LLAMA3 set up
1. Download from https://ollama.com/
2. Install
3. Click here http://localhost:11434/ to check if it’s running (it should appear 'Ollama is running')
4. Run in a terminal: `ollama run llama3`

If u dont have: `pip install langchain-community`

In [8]:
from langchain_community.llms import Ollama
import re

llm = Ollama(model="llama3")
sentences = ["I have tomatoes, onions, and garlic. What can I cook with them?",
             "What dishes can I make with pasta, spinach, and cheese?",
             "I found some shrimp, bell peppers, and mushrooms in the fridge. Any recipe suggestions?",
             "My pantry has rice, beans, and tomatoes. What's a simple yet delicious meal I can prepare?",
             "I have eggs, bacon, and bread. How can I turn these into a tasty breakfast?",
             "What can I whip up with ground beef, bell peppers, and onions?",
             "I bought salmon, asparagus, and lemon. What's a good recipe for a healthy dinner?",
             "I've got tofu, broccoli, and soy sauce. Any suggestions for a vegetarian stir-fry?",
             "What desserts can I make with flour, sugar, and chocolate?",
             "I'm craving something sweet. What can I make with apples, cinnamon, and oats?"]


the LLM approach could look like:
prompt:
List all food ingredients from the following text {{user prompt}

In [14]:

def extractIngredients(sentence):
    output = ''
    prompt = f"Output the ingredients from the following sentence without any comments, separated only by white spaces: {sentence}"
    for chunks in llm.stream(prompt):
        output += chunks

    pattern = r'\b[a-zA-Z]+\b'  # Simple pattern to match words
    ingredients = re.findall(pattern, output.lower())
    return list(ingredients)


In [16]:
for sentence in sentences:
    print(f"Sentence:\n{sentence}\nIngredients:\n{extractIngredients(sentence)}\n")

Sentence:
I have tomatoes, onions, and garlic. What can I cook with them?
Ingredients:
['tomatoes', 'onions', 'garlic']

Sentence:
What dishes can I make with pasta, spinach, and cheese?
Ingredients:
['pasta', 'spinach', 'cheese']

Sentence:
I found some shrimp, bell peppers, and mushrooms in the fridge. Any recipe suggestions?
Ingredients:
['shrimp', 'bell', 'peppers', 'mushrooms']

Sentence:
My pantry has rice, beans, and tomatoes. What's a simple yet delicious meal I can prepare?
Ingredients:
['rice', 'beans', 'tomatoes']

Sentence:
I have eggs, bacon, and bread. How can I turn these into a tasty breakfast?
Ingredients:
['eggs', 'bacon', 'bread']

Sentence:
What can I whip up with ground beef, bell peppers, and onions?
Ingredients:
['ground', 'beef', 'bell', 'peppers', 'onions']

Sentence:
I bought salmon, asparagus, and lemon. What's a good recipe for a healthy dinner?
Ingredients:
['salmon', 'asparagus', 'lemon']

Sentence:
I've got tofu, broccoli, and soy sauce. Any suggestions f